In [3]:
import os

import numpy as np
import pandas as pd
import plotly.express as px

os.environ["OMP_NUM_THREADS"] = '4'
from sklearn.cluster import KMeans

In [ ]:
df = pd.read_hdf(r"C:\Users\quicken\Code\Ambros_analysis\EBBS_workshop\")
df = df.iloc[6000:7000].reset_index(drop=True)

df = df.sort_index(axis=1)
#df = df.interpolate("linear")

scorer = df.columns.levels[0][0]
individuals = df.columns.levels[1].to_list()
bodyparts = df.columns.levels[2].to_list()

In [21]:
df

scorer      DLC_resnet152_11_2022_ML_DLC_PROJECTNov21shuffle1_650000           \
individuals                                               black_tail            
bodyparts                                                   dorsal_1            
coords                                                    likelihood        x   
0                                                        0.999        942.374   
1                                                        1.000        940.995   
2                                                        1.000        939.688   
3                                                        1.000        939.421   
4                                                        1.000        939.421   
..                                                         ...            ...   
995                                                      1.000        146.737   
996                                                      1.000        147.250   
997                                                      1.000        147.881   
998                                                      1.000        150.946   
999                                                      1.000        153.843   

scorer                                                                 \
individuals                                                             
bodyparts              dorsal_2                     dorsal_3            
coords             y likelihood        x        y likelihood        x   
0            408.023        1.0  949.104  437.278        1.0  958.206   
1            403.480        1.0  947.790  432.560        1.0  956.703   
2            399.326        1.0  946.711  428.802        1.0  956.502   
3            393.990        1.0  946.140  420.246        1.0  956.250   
4            387.773        1.0  946.140  416.186        1.0  956.250   
..               ...        ...      ...      ...        ...      ...   
995          324.616        1.0  142.515  354.378        1.0  142.573   
996          317.350        1.0  142.589  348.861        1.0  142.922   
997          310.228        1.0  144.718  341.221        1.0  145.653   
998          301.960        1.0  147.145  335.267        1.0  149.103   
999          294.690        1.0  148.887  330.774        1.0  149.449   

scorer                           ...                                    \
individuals                      ... non-marked                          
bodyparts              dorsal_4  ...  right_ear shoulder_left            
coords             y likelihood  ...          y    likelihood        x   
0            469.613        1.0  ...    378.612           1.0  530.152   
1            461.311        1.0  ...    379.581           1.0  530.152   
2            458.613        1.0  ...    381.063           1.0  530.317   
3            452.833        1.0  ...    382.790           1.0  530.361   
4            445.224        1.0  ...    384.239           1.0  530.895   
..               ...        ...  ...        ...           ...      ...   
995          387.453        1.0  ...    203.201           1.0  954.014   
996          380.836        1.0  ...    202.729           1.0  954.065   
997          375.095        1.0  ...    197.062           1.0  955.115   
998          368.142        1.0  ...    194.004           1.0  957.794   
999          361.930        1.0  ...    187.950           1.0  958.651   

scorer                                                                      \
individuals                                                                  
bodyparts            shoulder_right                     tail_base            
coords             y     likelihood         x        y likelihood        x   
0            326.605          1.000   521.849  396.584        1.0  411.776   
1            329.886          1.000   516.267  397.442        1.0  411.275   
2            329.962          1.000   513.038  397.606        1.0  410.369   
3            330.538          1

In [8]:
pip install tables

  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
   ---------------------------------------- 0.0/6.4 MB ? eta -:--:--
   --- ------------------------------------ 0.5/6.4 MB 15.2 MB/s eta 0:00:01
   -------- ------------------------------- 1.4/6.4 MB 17.5 MB/s eta 0:00:01
   ----------------- ---------------------- 2.8/6.4 MB 22.0 MB/s eta 0:00:01
   ------------------------------ --------- 4.9/6.4 MB 28.6 MB/s eta 0:00:01
   ---------------------------------------  6.4/6.4 MB 34.2 MB/s eta 0:00:01
   ---------------------------------------- 6.4/6.4 MB 25.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------  2.2/2.2 MB 71.3 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 47.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/146.2 kB 

In [5]:
arr_x = df.loc[:, (scorer, individuals[0], bodyparts, ["x", "y"])].values[:,::2]
arr_y = df.loc[:, (scorer, individuals[0], bodyparts, ["x", "y"])].values[:,1::2]

mean_x = arr_x.mean(axis=1)
mean_y = arr_y.mean(axis=1)

# die mitte der maus wird als center angesehen

centered_x = -(arr_x - mean_x[:, np.newaxis])
centered_y = arr_y - mean_y[:, np.newaxis]

In [6]:
# fix the position of the animal direction wise
# so behaviors that are the same but move into different directions will now be completly similar

nose_ind = bodyparts.index("nose")
coords = np.array([centered_x[0], centered_y[0]]).T
animal_vec = coords[nose_ind] - coords.mean(axis=0)
ref_vec = np.array([0,150]) - coords.mean(axis=0)
dot_product = np.dot(ref_vec, animal_vec)
magnitude_AB = np.linalg.norm(ref_vec)
magnitude_BC = np.linalg.norm(animal_vec)
angle = np.arccos(dot_product / (magnitude_AB * magnitude_BC))

In [7]:
rotated = []

for i in range(len(df)):
    nose_ind = bodyparts.index("nose")
    coords = np.array([centered_x[i], centered_y[i]]).T
    animal_vec = coords[nose_ind] - coords.mean(axis=0)
    ref_vec = np.array([0,150]) - coords.mean(axis=0)

    dot_product = np.dot(ref_vec, animal_vec)
    magnitude_AB = np.linalg.norm(ref_vec)
    magnitude_BC = np.linalg.norm(animal_vec)
    angle = np.arccos(dot_product / (magnitude_AB * magnitude_BC))
    A = coords.T
    if coords[nose_ind][0] < 0:
        theta = -angle
    else:
        theta = angle
    rotate = np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta),  np.cos(theta)]
    ])

    out = (rotate @ A).T
    
    rotated.append(out)
rotated = np.round(np.concatenate(rotated)).astype(int)

In [8]:
# Plot rotation
frame = 100

fig1 = px.scatter(
    x=centered_x[frame], 
    y=centered_y[frame], 
    color=bodyparts, 
    width=600, 
    height=600, 
    range_x=[-150, 150], 
    range_y=[-150, 150]
)
fig2 = px.scatter(
    x=rotated[frame*17:frame*17+17, 0], 
    y=rotated[frame*17:frame*17+17, 1],
    color=bodyparts, 
    width=600, 
    height=600, 
    range_x=[-150, 150], 
    range_y=[-150, 150]
)

fig1.show()
fig2.show()

# Make animatation
bps_col = bodyparts * len(df)
frame_col = sorted(list(range(len(df))) * len(bodyparts))

plot_df = pd.DataFrame({"x": rotated[:, 0], "y": rotated[:, 1]})
plot_df["bp"] = bps_col
plot_df["frame"] = frame_col

fig = px.scatter(
    plot_df.iloc[550*17:700*17], 
    x="x", 
    y="y", 
    color="bp", 
    animation_frame="frame", 
    width=600, height=600, 
    range_x=[-150, 150], 
    range_y=[-150, 150]
)
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 50
fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 20
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [9]:
X = rotated.reshape(-1, 34)

kmeans = KMeans(n_clusters=3, random_state=42, n_init="auto").fit(X)
kmeans.labels_

dfs = []

for i in range(len(np.unique(kmeans.labels_))):
    x = X[kmeans.labels_ == i][:, ::2].mean(axis=0)
    y = X[kmeans.labels_ == i][:, 1::2].mean(axis=0)
    dfs.append(pd.DataFrame({"x":x, "y":y, "bps":bodyparts, "cluster":str(i)}))

centered_df = pd.concat(dfs).reset_index(drop=True)

fig = px.scatter(
    centered_df, 
    x="x", 
    y="y", 
    color="bps", 
    facet_row="cluster", 
    width=400, 
    height=1000, 
    range_x=[-150, 150], 
    range_y=[-150, 150]
)
fig.update_traces(marker=dict(size=5))

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'bps=dorsal_1<br>cluster=0<br>x=%{x}<br>y=%{y}<extra></extra>',
              'legendgroup': 'dorsal_1',
              'marker': {'color': '#636efa', 'size': 5, 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'dorsal_1',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': {'bdata': 'y7Isy7IsF0A=', 'dtype': 'f8'},
              'xaxis': 'x3',
              'y': {'bdata': 'x3Ecx3EcK0A=', 'dtype': 'f8'},
              'yaxis': 'y3'},
             {'hovertemplate': 'bps=dorsal_1<br>cluster=1<br>x=%{x}<br>y=%{y}<extra></extra>',
              'legendgroup': 'dorsal_1',
              'marker': {'color': '#636efa', 'size': 5, 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'dorsal_1',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': {'bdata': 'X2uO/8ANAcA=', 'dtype': 'f8'},
              'xaxis': 'x2',
              'y': {'bdata': 'kjZFZJ7bL0A=', 'dtype': 'f8'},
              'yaxis': 'y2'},
             {'hovertemplate': 'bps=dorsal_1<br>cluster=2<br>x=%{x}<br>y=%{y}<extra></extra>',
              'legendgroup': 'dorsal_1',
              'marker': {'color': '#636efa', 'size': 5, 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'dorsal_1',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': {'bdata': 'Gc1lNJfRJMA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'EN9BfAfxnb8=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'bps=dorsal_2<br>cluster=0<br>x=%{x}<br>y=%{y}<extra></extra>',
              'legendgroup': 'dorsal_2',
              'marker': {'color': '#EF553B', 'size': 5, 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'dorsal_2',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': {'bdata': 'Pc/zPM/z/D8=', 'dtype': 'f8'},
              'xaxis': 'x3',
              'y': {'bdata': 'TdM0TdO0LcA=', 'dtype': 'f8'},
              'yaxis': 'y3'},
             {'hovertemplate': 'bps=dorsal_2<br>cluster=1<br>x=%{x}<br>y=%{y}<extra></extra>',
              'legendgroup': 'dorsal_2',
              'marker': {'color': '#EF553B', 'size': 5, 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'dorsal_2',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': {'bdata': 'k7QpIvPc/r8=', 'dtype': 'f8'},
              'xaxis': 'x2',
              'y': {'bdata': 'N0VkntvRLcA=', 'dtype': 'f8'},
              'yaxis': 'y2'},
             {'hovertemplate': 'bps=dorsal_2<br>cluster=2<br>x=%{x}<br>y=%{y}<extra></extra>',
              'legendgroup': 'dorsal_2',
              'marker': {'color': '#EF553B', 'size': 5, 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'dorsal_2',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': {'bdata': 'V1JbSW0lEcA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'WE9hPYX1NMA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'bps=dorsal_3<br>cluster=0<br>x=%{x}<br>y=%{y}<extra></extra>',
              'legendgroup': 'dorsal_3',
              'marker': {'color': '#00cc96', 'size': 5, 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'dorsal_3',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': {'bdata': 'KYqiKIqiEcA=', 'dtype': 'f8'},
              'xaxis': 'x3',
              'y': {'bdata': 'u67ruq4LRsA=', 'dtype': 'f8'},
              'yaxis': 'y3'},
             {'hovertemplate': 'bps=dorsal_3<br>cluster=1